In [ ]:
# This file contains all the main external libs we'll use
from fastai.imports import * #used for fastai
from IPython import display #used to display media in notebook
import soundfile as sf #used to load sound files
import matplotlib.pyplot as plt #used to plot in notebook
import scipy
from scipy.signal import hann
from scipy.fftpack import rfft

import pandas as pd
from fastai.data.all import *
from fastai.vision.all import *
import librosa

### Open the metadata file

Audio loading based on: https://github.com/limazix/audio-processing/blob/master/audio_classifier_fastai.ipynb

In [ ]:
def plot_spectrogram(spec, title=None, ylabel="freq_bin", aspect="auto", xmax=None):
    fig, axs = plt.subplots(1, 1)
    axs.set_title(title or "Spectrogram (db)")
    axs.set_ylabel(ylabel)
    axs.set_xlabel("frame")
    im = axs.imshow(librosa.power_to_db(spec), origin="lower", aspect=aspect)
    if xmax:
        axs.set_xlim((0, xmax))
    fig.colorbar(im, ax=axs)
    plt.show(block=False)
    
def to_specgram(audio_file, n_fft=256):
    waveform, sample_rate = torchaudio.load(audio_file)
    transform = torchaudio.transforms.Spectrogram(n_fft=n_fft)
    spectrogram = transform(waveform)
    return spectrogram[0]

class SpecgramTransform(Transform):
    def __init__(self): self.aug = to_specgram
    def encodes(self, audio_file: Path):
        aug_img = self.aug(audio_file)
        return aug_img

files = get_files(path)
file = files[0]
s = to_specgram(file)
plot_spectrogram(s, title='Example to see if data loaded')

In [ ]:
path = Path(DATA_ROOT_DIR)
tfm = SpecgramTransform()
dblock = DataBlock(
    blocks    = (TransformBlock, CategoryBlock),
    get_items = get_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms = [ tfm ]
)
dls = dblock.dataloaders(path)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)